<a href="https://colab.research.google.com/github/SLCFLAB/Data-Science-Python/blob/main/Day%2011/11_1.CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN

출처: https://github.com/SLCFLAB/Fintech2022

- 선형신경망의 경우, pixel을 펴서 학습시키기 때문에 공간적 구조를 잃어버릴 수 있음. 
- 또한, 저장되어야 하는 parameter가 매우 많음.
- 이미지 처리를 위해 CNN 모델이 만들어짐.
- 필터의 연산 활용함.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision.utils
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import numpy as np
import random
import os

import matplotlib.pyplot as plt
%matplotlib inline

## Load data

In [ ]:
train_data = dsets.MNIST(root='data/',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)

test_data = dsets.MNIST(root='data/',
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

In [ ]:
batch_size = 100 # 한번에 할 연산량

train_loader = DataLoader(dataset=train_data,
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(dataset=test_data,
                         batch_size=batch_size,
                         shuffle=False)

## Define model

- CNN 모델은 일반적으로 covolution과 pooling layer로 구성되어 있음(보통 그 사이에 ReLU layer을 넣지만, 최근 연구결과 순서를 바꿔도 됨이 밝혀짐)
- filter을 이용해 연산을 하면 크기가 줄어듦, padding을 통해 이를 해결할 수 있음.
- pooling을 통해 중요한 정보만 남게 할 수 있음. 보통 max pooling을 사용하며, 예를들어 2*2 max pooling을 하면 크기는 반으로 줄어듦.
- stride를 통해 건너뛰면서 연산을 할 수도 있음.


모델 만들 때 크기 계산 필수적임!!!
- input : N * N
- padding : P
- stride : S
- Filter : F * F

### output = {[(N+2P-F)/S] + 1} * {[(N+2P-F)/S] + 1} 
여기서 []는 가우스기호

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv_layer = nn.Sequential(
            nn.Conv2d(1, 16, 5), #[100,16,24,24]
            nn.ReLU(),
            nn.Conv2d(16, 32, 5),#[100,32,20,20]
            nn.ReLU(),
            nn.MaxPool2d(2, 2),#100,32,10,10
            nn.Conv2d(32, 64, 5),#100,64,6,6
            nn.ReLU(),
            nn.MaxPool2d(2, 2)#100,64,3,3 -> 여기서 batch 빼고 나머지 64*3*3
        )
        
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3, 100), #여기서 64*3*3 -> 100
            nn.ReLU(),
            nn.Linear(100, 10)#100 -> 10
        )       
        
    def forward(self,x):
        out = self.conv_layer(x)
        out = out.view(-1,64*3*3)
        out = self.fc_layer(out)

        return out

In [ ]:
model = CNN().cuda() ## GPU 사용

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

## Train model

In [ ]:
num_epochs = 3

In [ ]:
for epoch in range(num_epochs):

    total_batch = len(train_data) // batch_size

    for i, (batch_images, batch_labels) in enumerate(train_loader):

        X = batch_images.cuda()
        Y = batch_labels.cuda()

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 300 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

Epoch [1/3], lter [300/600], Loss: 2.2847
Epoch [1/3], lter [600/600], Loss: 1.9827
Epoch [2/3], lter [300/600], Loss: 0.4843
Epoch [2/3], lter [600/600], Loss: 0.2493
Epoch [3/3], lter [300/600], Loss: 0.1727
Epoch [3/3], lter [600/600], Loss: 0.1895


## Test model

In [ ]:
correct = 0
total = 0

for images, labels in test_loader:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Accuracy of test images: %f %%' % (100 * float(correct) / total))

Accuracy of test images: 94.590000 %
